# **Linear Regression - Life Expectancy Prediction**

En este notebook se entrena un modelo de **Regresión Lineal** para predecir la esperanza de vida a partir del dataset procesado (en el notebook `EDA_processed.ipynb`).  
La regresión lineal se utiliza como un modelo **baseline**, ya que es sencillo de interpretar y sirve como referencia frente a modelos más complejos.

Del EDA_processed.ipynb exportamos dos datasets:

- features_no_scaling.csv → para árboles, Random Forest, XGBoost.

- features_scaled.csv → para modelos sensibles a la escala (Regresión Lineal, KNN, SVM...).
Además del target_y.csv.

## **Paso 1. Importación de librerías y carga de datasets procesados**

**Sobre el dataset usado:**
En este notebook trabajamos con **Regresión Lineal**, un modelo sensible a la magnitud de las variables.  
Por eso cargamos el dataset **escalado** (`features_scaled.csv`), generado en el notebook `EDA_processed.ipynb` con `StandardScaler`.  

Esto asegura que todas las variables tengan media 0 y varianza 1, evitando que variables con unidades más grandes (por ejemplo PIB o gasto sanitario) dominen a otras más pequeñas.

In [ ]:
# ===================================
# 1. Importar librerías
# ===================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Visualización más estética
plt.style.use("seaborn-v0_8")
sns.set(font_scale=1.2)

# ===================================
# Cargar datasets procesados
# ===================================
X_scaled = pd.read_csv("../data/processed/features_scaled.csv")
y = pd.read_csv("../data/processed/target_y.csv").squeeze()  # convertir en Serie



## **Paso 2. División en entrenamiento y validación**

En este bloque:

- Separamos el dataset en **conjunto de entrenamiento (80%)** y **conjunto de validación (20%)**.  
Esto nos permite entrenar el modelo con una parte de los datos y luego evaluar su rendimiento en datos que no ha visto antes.


De esta manera, podremos comparar los algoritmos de forma justa y evitar errores de entrenamiento.

- x = todas las columnas salvo la variable objetivo.
- y = la variable objetivo.
- train_test_split divide los datos en:
  - x_train, y_train (80%) → para entrenar el modelo.
  - x_valid, y_valid (20%) → para validar el modelo. 
  - random_state=42 → asegura que la división sea reproducible.

Este paso es obligatorio para poder comparar modelos en condiciones.

In [ ]:
# ===================================
# 2. División train/validación
# ===================================
X_train, X_valid, y_train, y_valid = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print("🔹 Dataset escalado")
print("Tamaño entrenamiento:", X_train.shape, y_train.shape)
print("Tamaño validación:", X_valid.shape, y_valid.shape)


## **Paso 3. Entrenamiento y evaluación del modelo**


En este bloque entrenamos el modelo de **Regresión Lineal** con el conjunto de entrenamiento y lo evaluamos con el conjunto de validación.  

1. **Entrenamiento (`fit`)**:  
   Ajustamos el modelo a los datos de entrenamiento (`X_train`, `y_train`).  
   Esto significa que el algoritmo calcula los coeficientes (pesos) de la ecuación lineal que mejor explican la relación entre las variables independientes y la esperanza de vida.

2. **Predicción (`predict`)**:  
   Usamos los datos de validación (`X_valid`) para que el modelo prediga valores de `y`.  
   Estos valores predichos (`preds`) se comparan con los valores reales (`y_valid`).

3. **Métricas de evaluación**:  
   - **RMSE (Root Mean Squared Error)**: error cuadrático medio en la misma escala que la variable objetivo. Cuanto más bajo, mejor.  
     Penaliza más los errores grandes, por eso es útil para detectar desviaciones fuertes.  
   - **MAE (Mean Absolute Error)**: error absoluto medio. Es más robusto frente a valores atípicos que el RMSE.  
     Representa, en promedio, cuánto se equivoca el modelo al predecir.  
   - **R² (Coeficiente de determinación)**: mide qué proporción de la variabilidad de la variable objetivo se explica con el modelo.  
     Su valor está entre 0 y 1 (aunque puede ser negativo si el modelo es muy malo). Valores cercanos a 1 indican un buen ajuste.

En resumen:  
- Un **RMSE y MAE bajos** indican que las predicciones son precisas.  
- Un **R² alto** indica que el modelo explica gran parte de la variabilidad en la esperanza de vida.


In [ ]:
# ===================================
# 3. Entrenar y evaluar modelo
# ===================================

lr = LinearRegression()
lr.fit(X_train, y_train)

preds = lr.predict(X_valid)



# Métricas
rmse = np.sqrt(mean_squared_error(y_valid, preds))
mae = mean_absolute_error(y_valid, preds)
r2 = r2_score(y_valid, preds)

print("📊 Linear Regression (con datos escalados)")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")
print(f"R²: {r2:.3f}")


# ===================================
# Función para guardar métricas estandarizadas
# ===================================
def save_results(model_name, y_true, y_pred, filepath):
    """
    Calcula métricas (RMSE, MAE, R²) y guarda en CSV estandarizado.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    results = pd.DataFrame([{
        "Modelo": model_name,
        "RMSE": rmse,
        "MAE": mae,
        "R²": r2
    }])

    results.to_csv(filepath, index=False)
    print(f"✅ Resultados de {model_name} guardados en {filepath}")
    return results



# ===================================
# Guardar métricas Linear Regression
# ===================================
results_lr = save_results(
    "Linear Regression",
    y_valid,
    preds,
    "../data/results_linear.csv"
)





## **Paso 4. Visualización de resultados**

Este gráfico muestra la relación entre los valores reales de la esperanza de vida y las predicciones del modelo.  
- Si el modelo fuera perfecto, todos los puntos caerían sobre la línea roja diagonal.  
- La dispersión alrededor de la línea refleja el error de predicción.  


**¿Por qué seguimos viendo puntos alejados de la recta si "limpiamos" Outliers en el EDA inicial?**

### Outliers ≠ todo punto lejos de la recta de regresión

En el **EDA** aplicamos un tratamiento de *outliers* usando reglas como **IQR (rango intercuartílico)** o **boxplots** sobre las variables.  

Eso sirve para detectar valores extremos en cada **variable independiente (features)** de forma **univariada**.  

Sin embargo, cuando ajustamos una **regresión lineal**, los *puntos alejados de la recta* que vemos son **residuos grandes** (casos donde el modelo no predijo bien).  

👉 No siempre esos puntos son *outliers* en las features: a veces son valores legítimos que simplemente no siguen la relación lineal.


### Qué puede estar pasando aquí
- Eliminamos los outliers univariados, pero **no necesariamente los outliers multivariados** (casos raros en combinación de varias variables).  
- La **regresión lineal es muy sensible**: aunque un dato no sea extremo en cada feature, puede ser influyente y alejarse de la recta.  


### Opciones que tenemos
1. **Revisar los residuos**: graficar `y_valid vs. preds` y un histograma de los residuos.  
   - Si vemos muchos puntos dispersos lejos, el modelo no está capturando bien la complejidad.  
2. **Usar métricas robustas (MAE)** además de RMSE, porque el RMSE se dispara con outliers.  
3. **Considerar modelos no lineales** (Decision Tree, Random Forest o XGBoost), que manejan mucho mejor estos casos.  


In [ ]:
# ===================================
# 4. Gráfico de comparación
# ===================================
plt.figure(figsize=(7,7))
sns.scatterplot(x=y_valid, y=preds, alpha=0.7)
plt.plot([y_valid.min(), y_valid.max()],
         [y_valid.min(), y_valid.max()],
         color="red", linestyle="--")
plt.title("Predicciones vs Valores Reales - Regresión Lineal")
plt.xlabel("Valor real (Life Expectancy)")
plt.ylabel("Predicción")
plt.show()


## **Paso 5. Importancia de características (coeficientes del modelo)**

🔎 En este gráfico vemos la importancia relativa de cada variable en el modelo:  
- Un **coeficiente positivo** implica que la variable está asociada con un aumento en la esperanza de vida.  
- Un **coeficiente negativo** indica relación inversa (a mayor valor de la variable, menor esperanza de vida).  
- Esto nos da **interpretabilidad** del modelo, aunque hay que tener cuidado con la multicolinealidad (cuando varias variables están muy correlacionadas entre sí).  


In [ ]:
# ===================================
# 5. Importancia de variables (coeficientes)
# ===================================
coef_df = pd.DataFrame({
    "Variable": X_train.columns,
    "Coeficiente": lr.coef_
}).sort_values(by="Coeficiente", ascending=False)

plt.figure(figsize=(8,10))
sns.barplot(
    x="Coeficiente",
    y="Variable",
    data=coef_df,
    hue="Variable",        # asignar la variable al hue
    dodge=False,           # evita duplicados en barras
    legend=False,          # no mostrar leyenda redundante
    palette="coolwarm"
)
plt.title("Coeficientes de la Regresión Lineal", fontsize=14)
plt.xlabel("Peso del coeficiente")
plt.ylabel("Variable")
plt.show()

# Mostrar top 10 coeficientes en tabla
coef_df.head(10)

## **Paso 6. Diagnóstico de residuos en Regresión Lineal**


Un **residuo** es la diferencia entre el valor real (`y_valid`) y la predicción del modelo (`preds`).  

Analizar los residuos es importante porque:  
- Si el modelo es adecuado, los residuos deberían distribuirse alrededor de **0**, sin patrones claros.  
- Residuos grandes indican observaciones mal ajustadas (posibles *outliers multivariados*).  
- Si hay un patrón sistemático (curvas, abanicos, etc.), significa que la relación **no es estrictamente lineal** y el modelo lineal no captura toda la complejidad.  

En los gráficos que veremos:  

- **Gráfico real vs. predicciones**: si los puntos se agrupan alrededor de la diagonal (línea roja; línea `y = x`), el modelo se ajusta o está prediciendo bien. Si hay dispersión indica mala capacidad de predicción. 

- **Histograma / distribución de residuos**: debería parecerse a una distribución normal centrada en 0 (centrados en 0) y con forma aproximadamente normal. Colas largas sugieren outliers o mala especificación.

- **Gráfico de residuos vs. predicciones**: los residuos deberían distribuirse o estar dispersos de manera aleatoria alrededor de 0 (sin curva o abanico). Si vemos un patrón (curva, abanico, etc.), indica que el modelo lineal no está capturando bien la relación.

Estos gráficos permiten detectar:  
- **Heterocedasticidad**: cuando los residuos aumentan o disminuyen con el valor de las predicciones.  
- **Outliers multivariados**: puntos con residuos extremadamente altos.  



In [ ]:
# ===================================
# 6. Diagnóstico de residuos
# ===================================

# Calcular residuos
residuos = y_valid - preds

#--- Configuración de estilo global ---
plt.rcParams.update({
    "figure.figsize": (7,5),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10
})

# --- Gráfico 1: Valores reales vs. predicciones ---
plt.figure()
sns.scatterplot(x=y_valid, y=preds, alpha=0.7, color="steelblue", edgecolor="k")
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], 'r--', linewidth=2)
plt.xlabel("Valores reales")
plt.ylabel("Predicciones")
plt.title("Valores reales vs. Predicciones")
plt.show()

# --- Gráfico 2: Histograma de residuos ---
plt.figure()
sns.histplot(residuos, bins=30, kde=True, color="steelblue")
plt.axvline(0, color="red", linestyle="--", linewidth=2)
plt.title("Distribución de residuos")
plt.xlabel("Residuos (y_real - y_pred)")
plt.show()

# --- Gráfico 3: Residuos vs. Predicciones ---
plt.figure()
sns.scatterplot(x=preds, y=residuos, alpha=0.7, color="steelblue", edgecolor="k")
plt.axhline(0, color="red", linestyle="--", linewidth=2)
plt.xlabel("Predicciones")
plt.ylabel("Residuo")
plt.title("Residuos vs. Predicciones")
plt.show()


# --- Chequeo estadístico ---
print("📊 Resumen de residuos:")
print(residuos.describe())

# --- Detectar outliers multivariados ---
umbral = 2 * residuos.std()  # criterio simple: 2 desviaciones estándar
outliers = residuos[np.abs(residuos) > umbral]
print(f"🔎 Posibles outliers multivariados detectados: {len(outliers)}")


## **Paso 7. Conclusión del notebook del algoritmo de Regresión Lineal**

# 📌 Conclusiones de la Regresión Lineal
- La regresión lineal ofrece una **primera aproximación** sencilla para este problema.  
- Funciona bien cuando la relación entre variables es aproximadamente lineal y no hay demasiados outliers.  
- Sus métricas (RMSE, MAE, R²) nos servirán de **baseline** para comparar con modelos más avanzados como Árboles de Decisión, Random Forest y XGBoost. 


- La **Regresión Lineal** se ha entrenado sobre el dataset ya procesado (outliers tratados, nulos imputados, variables codificadas y escaladas).  
- Hemos obtenido métricas de rendimiento razonables que servirán como **baseline**.  
- Visualizamos las predicciones frente a los valores reales, comprobando la capacidad predictiva del modelo.  
- Identificamos las variables más influyentes mediante sus coeficientes.  
- A partir de aquí, podremos comparar este modelo con otros más complejos (árboles, Random Forest, XGBoost, Ridge, Lasso, ElasticNet, KNN).  

✅ Este notebook queda como **referencia inicial** para nuestro proyecto.  

